<h1>autobuildicdb NPD - Build Text Dictionaries</h1><br>

In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelFile
from pandas import ExcelWriter

import pyodbc
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, delete, insert, select, func, sql
import urllib


# Change Pandas display settings to show all columns
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
#pd.set_option('display.max_rows', 500)

In [2]:
# Read NPD 'wellbore_formation_top' Excel sheet to Pandas dataframe

df_wellfmtop = pd.read_excel('https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/wellbore_formation_top&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=EXCEL&Top100=false&IpAddress=91.109.232.40&CultureCode=en')

# Print column titles
print("NPD wellbore formation top columns:")
print(list(df_wellfmtop.columns))

NPD wellbore formation top columns:
['Wellbore name', 'Top depth [m]', 'Bottom depth [m]', 'Lithostrat. unit', 'Level', 'Lithostrat. unit, parent', 'NPDID wellbore', 'NPDID lithostrat. unit', 'NPDID parent lithostrat. unit', 'Date updated', 'Date sync NPD']


In [3]:
df_wellfmtop.head()

,Wellbore name,Top depth [m],Bottom depth [m],Lithostrat. unit,Level,"Lithostrat. unit, parent",NPDID wellbore,NPDID lithostrat. unit,NPDID parent lithostrat. unit,Date updated,Date sync NPD
0,9/8-1,1597.0,1625.0,BLODØKS FM,FORMATION,SHETLAND GP,145,13,143.0,2019-10-03,04.03.2020
1,9/8-1,1922.0,2109.0,SANDNES FM,FORMATION,VESTLAND GP,145,139,186.0,2019-10-03,04.03.2020
2,9/8-1,1777.0,1922.0,BOKNFJORD GP,GROUP,None,145,15,NaN,2019-10-03,04.03.2020
3,9/8-1,2109.0,2176.0,ZECHSTEIN GP,GROUP,None,145,194,NaN,2019-10-03,04.03.2020
4,9/8-1,1825.0,1922.0,EGERSUND FM,FORMATION,BOKNFJORD GP,145,30,15.0,2019-10-03,04.03.2020


In [4]:
print('Lithostratigraphy data for {} unique wells'.format(df_wellfmtop['Wellbore name'].nunique()))

Lithostratigraphy data for 1780 unique wells


<h3>Connect to database</h3>

In [5]:
# Configure and connect to SQL Server database

params = 'DRIVER={ODBC Driver 13 for SQL Server};' \
         'SERVER=5SQFPQ2\SQLEXPRESS;' \
         'PORT=1433;' \
         'DATABASE=Test3;' \
         'Trusted_Connection=yes;'
            
params = urllib.parse.quote_plus(params)

engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params, echo = True)

metadata = MetaData()
  
connection = engine.connect()
  
#Create Table objects
datalithostrat = Table('DATA_Lithostrat', metadata, autoload=True, autoload_with=engine)
symboldictionary = Table('SYMBOLDICTIONARY', metadata, autoload=True, autoload_with=engine)
symbols = Table('SYMBOLS', metadata, autoload=True, autoload_with=engine)

connection = engine.connect()

2020-03-04 21:58:51,935 INFO sqlalchemy.engine.base.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2020-03-04 21:58:51,935 INFO sqlalchemy.engine.base.Engine ()
2020-03-04 21:58:51,937 INFO sqlalchemy.engine.base.Engine SELECT schema_name()
2020-03-04 21:58:51,937 INFO sqlalchemy.engine.base.Engine ()
2020-03-04 21:58:51,940 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-04 21:58:51,941 INFO sqlalchemy.engine.base.Engine ()
2020-03-04 21:58:51,943 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS NVARCHAR(60)) AS anon_1
2020-03-04 21:58:51,944 INFO sqlalchemy.engine.base.Engine ()
2020-03-04 21:58:51,954 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME], [INFORMATION_SCHEMA].[COLUMNS].[IS_NULLABLE], [INFORMATION_SCHEMA].[COLUMNS].[DATA_TYPE], [INFORMATION_SCHEMA].[C

2020-03-04 21:58:52,286 INFO sqlalchemy.engine.base.Engine ('SYMBOLDICTIONARY', 'dbo')
2020-03-04 21:58:52,322 INFO sqlalchemy.engine.base.Engine select ind_col.index_id, ind_col.object_id, col.name from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=CAST(? AS NVARCHAR(max)) and sch.name=CAST(? AS NVARCHAR(max))
2020-03-04 21:58:52,323 INFO sqlalchemy.engine.base.Engine ('SYMBOLDICTIONARY', 'dbo')
2020-03-04 21:58:52,358 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME], [INFORMATION_SCHEMA].[COLUMNS].[IS_NULLABLE], [INFORMATION_SCHEMA].[COLUMNS].[DATA_TYPE], [INFORMATION_SCHEMA].[COLUMNS].[ORDINAL_POSITION], [INFORMATION_SCHEMA].[COLUMNS].[CHARACTER_MAXIMUM_L

In [6]:
selected_user_id = 1

In [7]:
from datetime import datetime

from xlrd import xldate

now = datetime.now()
now

timestampStr = now.strftime("%Y-%m-%d %H:%M:%S.%f")
print('Current Timestamp : ', timestampStr)

# Note that dbo.SYMBOLS and dbo.SYMBOLDICTIONARY expect datetime format,
# so no need to convert datetime to OLE automation date format.

Current Timestamp :  2020-03-04 21:58:52.695725


In [8]:
# Create a function to select all rows from table

def sqlselect_rows(tablename):
    
    select_stmt = select(tablename)
    result = connection.execute(select_stmt)
    
    # Equivalent to:
    # result = engine.execute('SELECT * FROM PROJECTS')

    for row in result:
        print(row)
    
    result.close()

<h3>Populate dbo.SYMBOLDICTIONARY</h3>

In [9]:
sql = " SELECT DISTINCT(dic_id) FROM dbo.SYMBOLDICTIONARY "
df = pd.read_sql_query(sql, engine)

# List unique values in dic_id
lst_dic_id = df["dic_id"].to_list()
print(lst_dic_id)

2020-03-04 21:58:52,715 INFO sqlalchemy.engine.base.Engine  SELECT DISTINCT(dic_id) FROM dbo.SYMBOLDICTIONARY 
2020-03-04 21:58:52,716 INFO sqlalchemy.engine.base.Engine ()
[-78, -77, -76, -75, -74, -73, -71, -70, -68, -66, -52, -48, -47, -46, 4, 5, 9, 12, 20, 21, 90, 94, 95, 96, 103, 114, 115, 118, 119, 126, 134, 135, 136, 137, 138, 32000, 32001, 32002, 32003]


In [10]:
# Find un-used dic_id
# Select a positive integer not already in list

for _ in range(3):
    m = range(1,len(lst_dic_id))
    n = min(set(m)-set(lst_dic_id))
    lst_dic_id.append(n)

In [11]:
# Check items have been appended
print(*lst_dic_id)

-78 -77 -76 -75 -74 -73 -71 -70 -68 -66 -52 -48 -47 -46 4 5 9 12 20 21 90 94 95 96 103 114 115 118 119 126 134 135 136 137 138 32000 32001 32002 32003 1 2 3


In [12]:
new_dic_id = lst_dic_id[-3:]

In [13]:
# Create dataframe for dbo.SYMBOLSDICTIONARY
# dic_id is a slice of the last 3 items from lst_dic_id
# Assign to original Lithostrat data types (110, etc) and new NPD Lithostrat data types (9011, etc)

symboldict_data = {'data_type' : [';110;9011;', ';112;9012;', ';113;9013;'],
                    'modified' : now,
                    'modifier' : selected_user_id,
                    'dic_id' : new_dic_id,
                    'dictionary_name' : ['NPD GROUPS', 'NPD FORMATIONS', 'NPD MEMBERS'],
                    'dic_type' : 0
                  }

#Create temp index, not sent to db
df_symboldict = pd.DataFrame(symboldict_data, index = ['dict_1', 'dict_2', 'dict_3'])
df_symboldict.head()

,data_type,modified,modifier,dic_id,dictionary_name,dic_type
dict_1,;110;9011;,2020-03-04 21:58:52.695725,1,1,NPD GROUPS,0
dict_2,;112;9012;,2020-03-04 21:58:52.695725,1,2,NPD FORMATIONS,0
dict_3,;113;9013;,2020-03-04 21:58:52.695725,1,3,NPD MEMBERS,0


In [14]:
# Append df_symboldict to dbo.SYMBOLDICTIONARY

df_symboldict.to_sql('SYMBOLDICTIONARY', engine, if_exists='append', index = False)

print('dbo.SYMBOLDICTIONARY')
sqlselect_rows([symboldictionary])

2020-03-04 21:58:52,776 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME], [INFORMATION_SCHEMA].[COLUMNS].[IS_NULLABLE], [INFORMATION_SCHEMA].[COLUMNS].[DATA_TYPE], [INFORMATION_SCHEMA].[COLUMNS].[ORDINAL_POSITION], [INFORMATION_SCHEMA].[COLUMNS].[CHARACTER_MAXIMUM_LENGTH], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_PRECISION], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_SCALE], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_DEFAULT], [INFORMATION_SCHEMA].[COLUMNS].[COLLATION_NAME] 
FROM [INFORMATION_SCHEMA].[COLUMNS] 
WHERE [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2020-03-04 21:58:52,777 INFO sqlalchemy.engine.base.Engine ('SYMBOLDICTIONARY', 'dbo')
2020-03-04 21:58:52,795 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-04 21:58:52,797 INFO sqlalchemy.engi

(36, ';5135;', datetime.datetime(2017, 3, 7, 14, 50, 43, 527000), 1, 135, 'ICS 2016 - ERAS', 0)
(37, ';5145;', datetime.datetime(2017, 3, 7, 14, 50, 43, 557000), 1, 136, 'ICS 2016 - PERIOD/SYSTEM', 0)
(38, ';5155;', datetime.datetime(2017, 3, 7, 14, 50, 43, 583000), 1, 137, 'ICS 2016 - SERIES / EPOCH', 0)
(39, ';66;', datetime.datetime(2017, 3, 7, 14, 50, 43, 617000), 1, 138, 'ICS 2016 - STAGE / AGE', 0)
(40, ';110;9011;', datetime.datetime(2020, 3, 4, 21, 58, 52, 697000), 1, 1, 'NPD GROUPS', 0)
(41, ';112;9012;', datetime.datetime(2020, 3, 4, 21, 58, 52, 697000), 1, 2, 'NPD FORMATIONS', 0)
(42, ';113;9013;', datetime.datetime(2020, 3, 4, 21, 58, 52, 697000), 1, 3, 'NPD MEMBERS', 0)


In [15]:
sql = ''' SELECT * FROM dbo.SYMBOLDICTIONARY '''
pd.read_sql_query(sql, engine)

2020-03-04 21:58:52,839 INFO sqlalchemy.engine.base.Engine  SELECT * FROM dbo.SYMBOLDICTIONARY 
2020-03-04 21:58:52,841 INFO sqlalchemy.engine.base.Engine ()


,pk_index,data_type,modified,modifier,dic_id,dictionary_name,dic_type
0,1,;,1899-12-30 00:00:00.000,0,-78,Spontaneous Potential,2
1,2,;201;,1899-12-30 00:00:00.000,0,4,Rock Description,1
2,3,;201;,1899-12-30 00:00:00.000,0,5,Porosity and Permeability,1
3,4,;2000;2010;2020;2030;,1899-12-30 00:00:00.000,0,9,Sample Symbols,1
4,5,;4010;,1899-12-30 00:00:00.000,0,12,Engineering Symbols,1
5,6,;700;,1899-12-30 00:00:00.000,0,20,Environment,0
6,7,;700;1000;,1899-12-30 00:00:00.000,0,21,Depositional Facies,0
7,8,;,1899-12-30 00:00:00.000,0,-46,Gamma Ray,2
8,9,;,1899-12-30 00:00:00.000,0,-47,Sonic,2
9,10,;,1899-12-30 00:00:00.000,0,-48,Porosity,2


<h3>Populate dbo.SYMBOLS</h3>

Create Text Dictionary entries for all unique lithostrat units, including their respective parent units.<br>

For testing purposes, filters 'NPD wellbore lithostrat' rows over smaller area of interest (e.g. "15/9" wells cover Volve Field).

In [16]:
df_wellfmtop.isnull().sum()
# 13842 of 36024 rows are null for 'Lithostrat. unit, parent', likely Groups

Wellbore name                        0
Top depth [m]                        0
Bottom depth [m]                     0
Lithostrat. unit                     0
Level                                0
Lithostrat. unit, parent         13867
NPDID wellbore                       0
NPDID lithostrat. unit               0
NPDID parent lithostrat. unit    13867
Date updated                         0
Date sync NPD                        0
dtype: int64

In [17]:
# # Use this cell to filter dataframe on part-wellname (Quad/Block) selection
# # Note: very few wells with NPD Members (e.g. "7220/7" )

# # String to be searched in start of string  
# search ="7720/7"

# # boolean series returned with False at place of NaN 
# bool_series = df_wellfmtop["Wellbore name"].str.startswith(search, na = False)

# #print(bool_series.value_counts())
  
# # Filtered dataframe 
# df_wellfmtop = df_wellfmtop[bool_series]
# df_wellfmtop.shape

In [18]:
df_wellfmtop

,Wellbore name,Top depth [m],Bottom depth [m],Lithostrat. unit,Level,"Lithostrat. unit, parent",NPDID wellbore,NPDID lithostrat. unit,NPDID parent lithostrat. unit,Date updated,Date sync NPD
0,9/8-1,1597.0,1625.0,BLODØKS FM,FORMATION,SHETLAND GP,145,13,143.0,2019-10-03,04.03.2020
1,9/8-1,1922.0,2109.0,SANDNES FM,FORMATION,VESTLAND GP,145,139,186.0,2019-10-03,04.03.2020
2,9/8-1,1777.0,1922.0,BOKNFJORD GP,GROUP,None,145,15,NaN,2019-10-03,04.03.2020
3,9/8-1,2109.0,2176.0,ZECHSTEIN GP,GROUP,None,145,194,NaN,2019-10-03,04.03.2020
4,9/8-1,1825.0,1922.0,EGERSUND FM,FORMATION,BOKNFJORD GP,145,30,15.0,2019-10-03,04.03.2020
...,...,...,...,...,...,...,...,...,...,...,...
36072,1/2-1,3275.0,3335.0,LISTA FM,FORMATION,ROGALAND GP,1382,95,131.0,2019-10-03,04.03.2020
36073,1/2-1,1777.0,3059.0,HORDALAND GP,GROUP,None,1382,67,NaN,2019-10-03,04.03.2020
36074,1/2-1,3121.0,3275.0,FORTIES FM,FORMATION,ROGALAND GP,1382,44,131.0,2019-10-03,04.03.2020
36075,1/2-1,3407.0,3514.0,EKOFISK FM,FORMATION,SHETLAND GP,1382,33,143.0,2019-10-03,04.03.2020


In [19]:
df_wellfmtop['Level'].unique()

array(['FORMATION', 'GROUP', 'MEMBER'], dtype=object)

In [20]:
# Create lists of each unqiue Gp, Fm and Mbr unit

lst_groups = list(df_wellfmtop[df_wellfmtop['Level'] == 'GROUP']['Lithostrat. unit'].unique())
print(len(lst_groups), 'unique Groups:')
print(lst_groups, '\n')

lst_formations = list(df_wellfmtop[df_wellfmtop['Level'] == 'FORMATION']['Lithostrat. unit'].unique())
print(len(lst_formations), 'unique Formations:')
print(lst_formations, '\n')

lst_members = list(df_wellfmtop[df_wellfmtop['Level'] == 'MEMBER']['Lithostrat. unit'].unique())
print(len(lst_members), 'unique Members:')
print(lst_members)

33 unique Groups:
['BOKNFJORD GP', 'ZECHSTEIN GP', 'HORDALAND GP', 'NORDLAND GP', 'ROGALAND GP', 'SHETLAND GP', 'VESTLAND GP', 'CROMER KNOLL GP', 'NO GROUP DEFINED', 'ROTLIEGEND GP', 'UNDEFINED GP', 'HEGRE GP', 'BASEMENT', 'TYNE GP', 'ADVENTDALEN GP', 'KAPP TOSCANA GP', 'SASSENDALEN GP', 'TEMPELFJORDEN GP', 'SOTBAKKEN GP', 'NYGRUNNEN GP', 'BJARMELAND GP', 'GIPSDALEN GP', 'BILLEFJORDEN GP', 'VIKING GP', 'GREY BEDS (INFORMAL)', 'BÅT GP', 'FANGST GP', 'RED BEDS (INFORMAL)', 'DUNLIN GP', 'STATFJORD GP', 'BRENT GP', 'NO DATA', 'FLADEN GP'] 

137 unique Formations:
['BLODØKS FM', 'SANDNES FM', 'EGERSUND FM', 'EKOFISK FM', 'LISTA FM', 'RØDBY FM', 'SELE FM', 'TOR FM', 'VÅLE FM', 'ÅSGARD FM', 'BØRGLUM UNIT', 'BALDER FM', 'HOD FM', 'BRYNE FM', 'SAUDA FM', 'FLEKKEFJORD FM', 'SMITH BANK FM', 'SKAGERRAK FM', 'TAU FM', 'SOLA FM', 'HIDRA FM', 'FISKEBANK FM', 'MAUREEN FM', 'FJERRITSLEV FM', 'TUXEN FM', 'GASSUM FM', 'UNDIFFERENTIATED', 'NO FORMAL NAME', 'VADE FM', 'VIDAR FM', 'ULA FM', 'FARSUND FM', 'M

In [21]:
# Find parent unit for each unique Gp, Fm, Mbr

# For each unique value in one column, get unique values in another column
# See https://stackoverflow.com/questions/48979604

group = df_wellfmtop.groupby('Lithostrat. unit')

df_symbols = group.apply(lambda x: x['Lithostrat. unit, parent'].unique())
df_symbols = df_symbols.apply(pd.Series)
df_symbols

,0
Lithostrat. unit,
ADVENTDALEN GP,None
AGAT FM,CROMER KNOLL GP
AKKAR MBR,FRUHOLMEN FM
ALGE MBR,HEKKINGEN FM
ALKE FM,HEGRE GP
...,...
ZECHSTEIN GP,None
ÅRE FM,BÅT GP
ÅSGARD FM,CROMER KNOLL GP


In [22]:
# Move 'Lithostrat. unit' from index to column
df_symbols = df_symbols.reset_index()

In [23]:
# Rename both coolumns to match dbo.SYMBOLS
df_symbols.columns = ['description', 'parent_unit']
df_symbols.head()

,description,parent_unit
0,ADVENTDALEN GP,None
1,AGAT FM,CROMER KNOLL GP
2,AKKAR MBR,FRUHOLMEN FM
3,ALGE MBR,HEKKINGEN FM
4,ALKE FM,HEGRE GP


In [24]:
#Create dic_id column based on level (Gp, Fm or Mbr)

def dicidnum(row):
   if row['description'] in lst_groups :
      return new_dic_id[0]
   if row['description'] in lst_formations :
      return new_dic_id[1]
   if row['description'] in lst_members :
      return new_dic_id[2]
   return 'Other'

#df_symbols.apply(lambda row: dicidnum(row), axis=1)
df_symbols['dic_id'] = df_symbols.apply(lambda row: dicidnum(row), axis=1)

#Sort dataframe by newly inserted dic_id
df_symbols.sort_values(by=['dic_id', 'description'], inplace=True)
df_symbols

#TO DO: Add new column to accomodate customised sorting with newer version?

,description,parent_unit,dic_id
0,ADVENTDALEN GP,None,1
8,BASEMENT,None,1
9,BILLEFJORDEN GP,None,1
10,BJARMELAND GP,None,1
13,BOKNFJORD GP,None,1
...,...,...,...
3,ALGE MBR,HEKKINGEN FM,3
79,KRABBE MBR,FRUHOLMEN FM,3
80,KRILL MBR,HEKKINGEN FM,3
95,MEILE MBR (INFORMAL),HEIMDAL FM,3


In [25]:
# Insert new columns to match dbo.SYMBOLS
# Only populate those necessary?

#df_symbols['lithology'] = None
#df_symbols['node'] = 0
#df_symbols['stretchx'] = 0
#df_symbols['stretchy'] = 0
df_symbols['modified'] = now # TO DO: CHECK THAT MILLISECONDS ARE NOT SHOWN INSIDE IC
df_symbols['modifier'] = 0
df_symbols['symbol_id'] = pd.Series(range(1000,len(df_symbols)+1001)) #TO DO: SYMBOL_ID NEEDS TO BE UNIQUE. SHOULD CHECK EXISTING DATA SET IN IC.
#df_symbols['youngest'] = 0
#df_symbols['oldest'] = 0
#df_symbols['width'] = 0
#df_symbols['height'] = 0
df_symbols['repeatable'] = False
df_symbols['back_trans'] = False
df_symbols['background'] = '8;8454143;0' # This is default yellow colour
#df_symbols[top_surface] = None
#df_symbols[base_surface] = None
#df_symbols['surface type'] = None
df_symbols['abbrev'] = df_symbols['description'].str[:3] # First 3 characters as abbrev
#df_symbols['f_wmf'] = None #TO DO: FIGURE OUT WHAT THESE TWO F_ COLUMNS ARE FOR
#df_symbols['f_definition_blob'] = None

In [26]:
df_symbols.sort_values(by='symbol_id')

,description,parent_unit,dic_id,modified,modifier,symbol_id,repeatable,back_trans,background,abbrev
0,ADVENTDALEN GP,None,1,2020-03-04 21:58:52.695725,0,1000,False,False,8;8454143;0,ADV
1,AGAT FM,CROMER KNOLL GP,2,2020-03-04 21:58:52.695725,0,1001,False,False,8;8454143;0,AGA
2,AKKAR MBR,FRUHOLMEN FM,3,2020-03-04 21:58:52.695725,0,1002,False,False,8;8454143;0,AKK
3,ALGE MBR,HEKKINGEN FM,3,2020-03-04 21:58:52.695725,0,1003,False,False,8;8454143;0,ALG
4,ALKE FM,HEGRE GP,2,2020-03-04 21:58:52.695725,0,1004,False,False,8;8454143;0,ALK
...,...,...,...,...,...,...,...,...,...,...
171,ZECHSTEIN GP,None,1,2020-03-04 21:58:52.695725,0,1171,False,False,8;8454143;0,ZEC
172,ÅRE FM,BÅT GP,2,2020-03-04 21:58:52.695725,0,1172,False,False,8;8454143;0,ÅRE
173,ÅSGARD FM,CROMER KNOLL GP,2,2020-03-04 21:58:52.695725,0,1173,False,False,8;8454143;0,ÅSG
174,ØRN FM,GIPSDALEN GP,2,2020-03-04 21:58:52.695725,0,1174,False,False,8;8454143;0,ØRN


In [27]:
df_symbols.shape

(176, 10)

In [28]:
# Append df_symbols to database
df_symbols.to_sql('SYMBOLS', engine, if_exists='append', index = False)
print('dbo.SYMBOLS')

2020-03-04 21:58:53,148 INFO sqlalchemy.engine.base.Engine SELECT [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA], [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_NAME], [INFORMATION_SCHEMA].[COLUMNS].[IS_NULLABLE], [INFORMATION_SCHEMA].[COLUMNS].[DATA_TYPE], [INFORMATION_SCHEMA].[COLUMNS].[ORDINAL_POSITION], [INFORMATION_SCHEMA].[COLUMNS].[CHARACTER_MAXIMUM_LENGTH], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_PRECISION], [INFORMATION_SCHEMA].[COLUMNS].[NUMERIC_SCALE], [INFORMATION_SCHEMA].[COLUMNS].[COLUMN_DEFAULT], [INFORMATION_SCHEMA].[COLUMNS].[COLLATION_NAME] 
FROM [INFORMATION_SCHEMA].[COLUMNS] 
WHERE [INFORMATION_SCHEMA].[COLUMNS].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[COLUMNS].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2020-03-04 21:58:53,149 INFO sqlalchemy.engine.base.Engine ('SYMBOLS', 'dbo')
2020-03-04 21:58:53,166 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-04 21:58:53,169 INFO sqlalchemy.engine.base.E

In [29]:
sql = ''' SELECT * FROM dbo.SYMBOLS '''
pd.read_sql_query(sql, engine)

2020-03-04 21:58:53,354 INFO sqlalchemy.engine.base.Engine  SELECT * FROM dbo.SYMBOLS 
2020-03-04 21:58:53,355 INFO sqlalchemy.engine.base.Engine ()


pk_index lithology  node  stretchx  stretchy                modified  modifier  symbol_id  dic_id           description  youngest  oldest  width  height  repeatable  back_trans    background top_surface base_surface   parent_unit surface_type           abbrev                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             